<a href="https://colab.research.google.com/github/janiosl/python.ds/blob/master/ml/ETL/ML_preproc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Carga dos dados originais

Inicialmente faremos a carga dos dados originais e sua separação entre características e classe alvo

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
#Definção dos títulos das colunas
headers = ['ESCT', 'NDEP', 'RENDA', 'TIPOR', 'VBEM', 'NPARC',
           'VPARC', 'TEL', 'IDADE', 'RESMS', 'ENTRADA', 'CLASSE']


#Carga do conjunto de treino
arquivo = 'https://raw.githubusercontent.com/MLRG-CEFET-RJ/ml-class/master/ml-t3/datasets/credtrain.txt'
data_train = pd.read_csv(arquivo, sep='\t', header=None, names=headers)


#Transoformação dos atributos e da classe alvo em matrizes
X_train_ = np.array(data_train.iloc[:, 0:11])
y_train_ = np.array(data_train['CLASSE'])


#Separação de treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_train_, y_train_,
                                                  train_size=0.2,
                                                  random_state=31)


#Carga do conjunto de teste
arquivo = 'https://raw.githubusercontent.com/MLRG-CEFET-RJ/ml-class/master/ml-t3/datasets/credtest.txt'
data_test = pd.read_csv(arquivo, sep='\t', header=None, names=headers)


#Transformação dos atributos e da classe alvo em matrizes
X_test = np.array(data_test.iloc[:, 0:11])
y_test = np.array(data_test['CLASSE'])

In [3]:
#Checagem rápida de parte dos dados carregados
print(data_train.head(),
      data_test.head(),
      X_train[0],
      y_train[0],
      sep='\n\n')

   ESCT  NDEP  RENDA  TIPOR  VBEM  ...  TEL  IDADE  RESMS  ENTRADA  CLASSE
0     1     0    360      0   313  ...    0     25     48        0       1
1     0     0    350      1   468  ...    0     33      6        0       1
2     0     0   1100      0   829  ...    0     56     48        0       1
3     0     0   3000      0   552  ...    1     31     60        0       1
4     1     0   1000      0   809  ...    0     24      7        0       1

[5 rows x 12 columns]

   ESCT  NDEP  RENDA  TIPOR  VBEM  ...  TEL  IDADE  RESMS  ENTRADA  CLASSE
0     0     2    500      1   618  ...    0     36      6        0       0
1     1     0    813      0   552  ...    0     43     48      119       1
2     3     0    350      0   488  ...    0     43      0        0       1
3     1     0   1530      0   381  ...    0     28     48        0       1
4     0     0    688      1   396  ...    0     49     72        0       1

[5 rows x 12 columns]

[  0   0 430   0 422  10  64   0  36   0   0]

1


In [4]:
data_train.describe()

,ESCT,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE
count,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,0.512000,0.126667,984.51200,0.445333,562.031333,8.392667,102.247333,0.134000,41.210667,31.528667,32.255333,0.476667
std,0.659909,0.572865,905.53292,0.497168,265.690646,3.609131,62.987325,0.340766,13.202266,52.662099,94.307614,0.499622
min,0.000000,0.000000,300.00000,0.000000,300.000000,1.000000,50.000000,0.000000,19.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,467.50000,0.000000,410.000000,6.000000,66.000000,0.000000,31.000000,6.000000,0.000000,0.000000
50%,0.000000,0.000000,650.00000,0.000000,490.000000,10.000000,83.000000,0.000000,39.000000,6.000000,0.000000,0.000000
75%,1.000000,0.000000,1200.00000,1.000000,618.000000,10.000000,118.000000,0.000000,52.000000,48.000000,0.000000,1.000000
max,3.000000,7.000000,8000.00000,1.000000,4000.000000,24.000000,711.000000,1.000000,70.000000,420.000000,1300.000000,1.000000


#Normalização dos dados

Como pode ser vistos nas saídas do final da seção as características estão em ordens de grandeza diferentes. Para muitos algoritmos este é um problema que pode levar a modelos inadequados. Para resolver este problema realizaremos a normalização dos dados.

In [5]:
#Criação do transformador para noramlização
from sklearn.preprocessing import StandardScaler
norm = StandardScaler()

In [6]:
#Normalização das características
X_train = norm.fit_transform(X_train)
X_test = norm.fit_transform(X_test)
X_val = norm.fit_transform(X_val)

In [7]:
#Checagem rápida de parte dos dados normalizados
print(X_train[0],
      X_val[0],
      X_test[0],
      sep='\n\n')

[-0.69652603 -0.23995691 -0.55588511 -0.78288136 -0.63954325  0.48186559
 -0.68331454 -0.33947258 -0.41404433 -0.61178962 -0.41850915]

[-0.79713982 -0.21807746 -0.78147173  1.07984489  0.2442632   0.98835168
 -0.7312085   2.46146776  0.97922561  0.58128115  1.97895427]

[-0.77125493  3.69396032 -0.49166148  1.10033085  0.14085423  0.44020867
 -0.27731176 -0.35630482 -0.38533555 -0.46487844 -0.29246359]


#Variáveis "Dummies" para tratamento de dados categóricos nominais

Algumas variáveis no conjunto de dados são discretas nominais, ou seja, os valores apresentados não representam uma contagem ordinal, mas categorias. Essa situação também pode gerar inconsistências na criação e treinamento de modelos. Para lidar com essa situação serão criadas variáveis *dummies*, ou seja, variáveis que representam cada categoria e assumem valores binárias (1 = verdadeiro / 0 = falso).

In [40]:
data_train.head()

,ESCT,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE
0,1,0,360,0,313,9,52,0,25,48,0,1
1,0,0,350,1,468,10,65,0,33,6,0,1
2,0,0,1100,0,829,9,125,0,56,48,0,1
3,0,0,3000,0,552,12,76,1,31,60,0,1
4,1,0,1000,0,809,12,111,0,24,7,0,1


In [41]:
#Conferência de valores da variável Estado Civil (ESCT)
print(data_train['ESCT'].head(10),
      data_train['ESCT'].describe(),
      sep='\n\n')

0    1
1    0
2    0
3    0
4    1
5    0
6    0
7    0
8    0
9    0
Name: ESCT, dtype: int64

count    1500.000000
mean        0.512000
std         0.659909
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         3.000000
Name: ESCT, dtype: float64


In [42]:
#Conferência dos valores possíveis da variável
set(data_train['ESCT'])

{0, 1, 2, 3}

Percebemos pela análise simplificada que as opções de valores para estado cívil estão registradas na base de dados como os valores 0, 1, 2 e 3. Para criação de variáveis *dummies* serão criadas $n-1$ variáveis binárias, onde $n$ é o número de categorias. Isso é feito porque a categoria $n$ ocorrerá quanto todas as demais tiverem valor 0 e, portanto, não precisa ser mapeada.

---

Para exemplificar a lógica consideremos o estado civil cuja categoria é 0. Podemos consultar o dataframe original da forma abaixo e obtemos um vetor de valores lógicos, na qual é apontado verdadeiro para os casos em que a categoria do estado civil é 0.

In [43]:
data_train['ESCT']==0

0       False
1        True
2        True
3        True
4       False
        ...  
1495    False
1496    False
1497     True
1498     True
1499     True
Name: ESCT, Length: 1500, dtype: bool

In [44]:
ESCT_0 = (data_train['ESCT']==0)
ESCT_0

0       False
1        True
2        True
3        True
4       False
        ...  
1495    False
1496    False
1497     True
1498     True
1499     True
Name: ESCT, Length: 1500, dtype: bool

Para criar as variáveis, basta usar esta mesma lógica para as categorias desejadas, transformando os valores lógicos em inteiros. Contudo, a biblioteca `pandas` possui um método denominado `get_dummies` que simplifica esta transformação. Aplicaremos abaixo o método no conjunto de dados.

In [45]:
pd.get_dummies(data=data_train,
               prefix='ESCT',
               columns=['ESCT'],
               drop_first=True)

,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE,ESCT_1,ESCT_2,ESCT_3
0,0,360,0,313,9,52,0,25,48,0,1,1,0,0
1,0,350,1,468,10,65,0,33,6,0,1,0,0,0
2,0,1100,0,829,9,125,0,56,48,0,1,0,0,0
3,0,3000,0,552,12,76,1,31,60,0,1,0,0,0
4,0,1000,0,809,12,111,0,24,7,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0,500,0,437,12,60,0,60,48,0,1,1,0,0
1496,0,2057,0,301,1,318,0,35,48,0,1,1,0,0
1497,0,570,0,511,12,71,1,28,144,0,1,0,0,0
1498,0,360,0,384,12,52,0,33,36,0,1,0,0,0


Veja que o atributo 'ESCT' foi substituído pelas variáveis novas e criado um data frame com os dados originais, exceto por esta substituição. Além disso, veja que foi definido o parâmetro `drop_first` com valor `True`, isso força o método a eliminar a primeira categoria, tornando a quantidade de *dummies* igual a $n-1$ pelos motivos explicados anteriormente nessa seção. Caso queira que todas as categorias estejam presentes no novo *data frame*, o parâmetro `drop_first` deve ser descartado, tendo em vista que seu valor padrão é `False`.

In [46]:
#Usando a lógica estuda para recarregar os dados e refazer a separação
data_train_ = pd.get_dummies(data=data_train,
               prefix='ESCT',
               columns=['ESCT'],
               drop_first=True)

data_train_.head()

,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE,ESCT_1,ESCT_2,ESCT_3
0,0,360,0,313,9,52,0,25,48,0,1,1,0,0
1,0,350,1,468,10,65,0,33,6,0,1,0,0,0
2,0,1100,0,829,9,125,0,56,48,0,1,0,0,0
3,0,3000,0,552,12,76,1,31,60,0,1,0,0,0
4,0,1000,0,809,12,111,0,24,7,0,1,1,0,0


In [47]:
#Usando a lógica estuda para recarregar os dados e refazer a separação
data_test_ = pd.get_dummies(data=data_test,
               prefix='ESCT',
               columns=['ESCT'],
               drop_first=True)

data_test_.head()

,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE,ESCT_1,ESCT_2,ESCT_3
0,2,500,1,618,10,85,0,36,6,0,0,0,0,0
1,0,813,0,552,4,119,0,43,48,119,1,1,0,0
2,0,350,0,488,12,66,0,43,0,0,1,0,0,1
3,0,1530,0,381,1,398,0,28,48,0,1,1,0,0
4,0,688,1,396,10,60,0,49,72,0,1,0,0,0


In [48]:
#Separação da variável algo não muda
##=======================================

#Transoformação dos atributos e da classe alvo em matrizes
y_train_ = np.array(data_train['CLASSE'])

#Transformação dos atributos e da classe alvo em matrizes
y_test = np.array(data_test['CLASSE'])

#Checagem rápida
print(y_train_[:10],
      y_test[:10],
      sep='\n\n')

[1 1 1 1 1 1 0 1 0 1]

[0 1 1 1 1 0 0 1 0 1]


In [49]:
#Separação dos atributos precisa atentar que não preservada a ordem das colunas
##=============================================================================

#Recuperação dos nomes das colunas
atributos = list(data_train_.columns)

#Checagem rápida
atributos

['NDEP',
 'RENDA',
 'TIPOR',
 'VBEM',
 'NPARC',
 'VPARC',
 'TEL',
 'IDADE',
 'RESMS',
 'ENTRADA',
 'CLASSE',
 'ESCT_1',
 'ESCT_2',
 'ESCT_3']

In [50]:
#Eliminação da colunas com a classe na lista de atributos
atributos.remove('CLASSE')

#Checagem rápida
atributos

['NDEP',
 'RENDA',
 'TIPOR',
 'VBEM',
 'NPARC',
 'VPARC',
 'TEL',
 'IDADE',
 'RESMS',
 'ENTRADA',
 'ESCT_1',
 'ESCT_2',
 'ESCT_3']

In [51]:
#Conferência do uso da lista de atributos para separação do conjunto de dados
data_train_.loc[:, atributos]

,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,ESCT_1,ESCT_2,ESCT_3
0,0,360,0,313,9,52,0,25,48,0,1,0,0
1,0,350,1,468,10,65,0,33,6,0,0,0,0
2,0,1100,0,829,9,125,0,56,48,0,0,0,0
3,0,3000,0,552,12,76,1,31,60,0,0,0,0
4,0,1000,0,809,12,111,0,24,7,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0,500,0,437,12,60,0,60,48,0,1,0,0
1496,0,2057,0,301,1,318,0,35,48,0,1,0,0
1497,0,570,0,511,12,71,1,28,144,0,0,0,0
1498,0,360,0,384,12,52,0,33,36,0,0,0,0


In [58]:
#Conferência do resumo estatístico do novo conjunto de dados
data_train_.loc[:, atributos].describe()

,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,ESCT_1,ESCT_2,ESCT_3
count,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,0.126667,984.51200,0.445333,562.031333,8.392667,102.247333,0.134000,41.210667,31.528667,32.255333,0.400667,0.018667,0.024667
std,0.572865,905.53292,0.497168,265.690646,3.609131,62.987325,0.340766,13.202266,52.662099,94.307614,0.490197,0.135390,0.155159
min,0.000000,300.00000,0.000000,300.000000,1.000000,50.000000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,467.50000,0.000000,410.000000,6.000000,66.000000,0.000000,31.000000,6.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,650.00000,0.000000,490.000000,10.000000,83.000000,0.000000,39.000000,6.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1200.00000,1.000000,618.000000,10.000000,118.000000,0.000000,52.000000,48.000000,0.000000,1.000000,0.000000,0.000000
max,7.000000,8000.00000,1.000000,4000.000000,24.000000,711.000000,1.000000,70.000000,420.000000,1300.000000,1.000000,1.000000,1.000000


In [53]:
#Uso da lista de atributos para separação do conjunto de dados

#Transoformação dos atributos e da classe alvo em matrizes
X_train_ = np.array(data_train_.loc[:, atributos])

#Transformação dos atributos e da classe alvo em matrizes
X_test = np.array(data_test_.loc[:, atributos])

In [54]:
#Separação de treino e validação não muda
##=======================================

#Separação de treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_train_, y_train_,
                                                  train_size=0.2,
                                                  random_state=31)

In [59]:
#Checagem rápida de parte dos dados carregados
print(X_train[:5],
      X_val[:5],
      X_test[:5],
      sep='\n\n')

[[   0  430    0  422   10   64    0   36    0    0    0    0    0]
 [   0 2000    1  387    9   63    0   52    6    0    1    0    0]
 [   0  700    1  770   12  104    0   45    6    0    0    0    0]
 [   0  990    0  531    6  118    0   30    0    0    0    0    0]
 [   0  705    0  409   10   63    0   35   48    0    0    0    0]]

[[   0  300    1  629   12   55    1   54   60  230    0    0    0]
 [   0  600    1  540   12   73    0   28    6    0    1    0    0]
 [   0  317    1  459   12   63    1   57  264    0    0    0    0]
 [   0  800    0  399    3  149    1   57    0    0    0    0    0]
 [   0 3000    0  360    1  376    0   43   48    0    1    0    0]]

[[   2  500    1  618   10   85    0   36    6    0    0    0    0]
 [   0  813    0  552    4  119    0   43   48  119    1    0    0]
 [   0  350    0  488   12   66    0   43    0    0    0    0    1]
 [   0 1530    0  381    1  398    0   28   48    0    1    0    0]
 [   0  688    1  396   10   60    0   49   